<a href="https://colab.research.google.com/github/kirank981/Project_space/blob/main/project_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependences


Install the necessary packages for PyTorch (torch and torchvision) and Flower (flwr) and pandas

In [1]:
!pip install -q flwr[simulation] torch torchvision matplotlib pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 25.2 MB/s eta 0:00:00


Import everything we need

In [2]:
from collections import OrderedDict
from typing import List, Tuple

from google.colab import drive
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
# from torchvision.datasets import CIFAR10



from sklearn.model_selection import train_test_split  # Import the train_test_split function
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

!pip install catboost
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# from tensorflow.keras.layers import Conv2D, Multiply, Input
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Multiply

import flwr as fl
from flwr.common import Metrics

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 3.0 MB/s eta 0:00:00
Training on cpu using PyTorch 2.0.1+cu118 and Flower 1.4.0


# Loading the data

Mounting drive

In [3]:
# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Setting the path to the location of the file

In [4]:
# Define the path to daily dataset folder
daily_dataset_path = Path('/content/drive/MyDrive/Federated learning implementation/dataset/dataset_archive/daily_dataset/daily_dataset')

# Define the path to daily dataset folder
weather_daily_dataset_path = Path('/content/drive/MyDrive/Federated learning implementation/dataset/dataset_archive/weather_daily_dataset.csv')


## Loading daily data
(of energy consumption)


In [5]:

# Initializing list to store dataframes
dfs = []

# Loop through the CSV files and reading them into dataframes
for i in range(111):
    filename = f'block_{i}.csv'
    df = pd.read_csv(daily_dataset_path / filename)
    dfs.append(df)

# Concatenating all the dataframes into a single dataframe
energy_daily_data = pd.concat(dfs, ignore_index=True)

# NRATM(Not Required At The Moment)
# # Group the data by LCLid and create a dictionary of dataframes
# grouped_data = dict(tuple(energy_daily_data.groupby('LCLid')))


Loading data using file name

In [6]:
# Loading data from a specific CSV file
specific_file_data = pd.read_csv(daily_dataset_path / 'block_0.csv')

# Displaying data
print(specific_file_data)

           LCLid         day  energy_median  energy_mean  energy_max  \
0      MAC000002  2012-10-12         0.1385     0.154304       0.886   
1      MAC000002  2012-10-13         0.1800     0.230979       0.933   
2      MAC000002  2012-10-14         0.1580     0.275479       1.085   
3      MAC000002  2012-10-15         0.1310     0.213688       1.164   
4      MAC000002  2012-10-16         0.1450     0.203521       0.991   
...          ...         ...            ...          ...         ...   
25569  MAC005492  2014-02-24         0.1690     0.175042       0.378   
25570  MAC005492  2014-02-25         0.1550     0.160792       0.545   
25571  MAC005492  2014-02-26         0.1490     0.178542       0.687   
25572  MAC005492  2014-02-27         0.1140     0.146167       0.478   
25573  MAC005492  2014-02-28         0.0880     0.088000       0.088   

       energy_count  energy_std  energy_sum  energy_min  
0                46    0.196034       7.098       0.000  
1                48

### Loading required data
Creating a DataFrame that have only the required data

In [7]:
selected_column = ['LCLid','energy_sum','day']
energy_daily_selected=energy_daily_data[selected_column]
# print(energy_daily_selected)

# Group the data by LCLid and create a dictionary of dataframes, allowing to access each dataframe separately using the LCLid as the key
grouped_data_selected = dict(tuple(energy_daily_selected.groupby('LCLid')))
# Display the data for 'MAC000002'
print('\n')
print('Data report of MAC000002')
print(grouped_data_selected['MAC000002'])




Data report of MAC000002
         LCLid  energy_sum         day
0    MAC000002       7.098  2012-10-12
1    MAC000002      11.087  2012-10-13
2    MAC000002      13.223  2012-10-14
3    MAC000002      10.257  2012-10-15
4    MAC000002       9.769  2012-10-16
..         ...         ...         ...
500  MAC000002      12.528  2014-02-24
501  MAC000002      11.826  2014-02-25
502  MAC000002      12.328  2014-02-26
503  MAC000002      20.518  2014-02-27
504  MAC000002       1.387  2014-02-28

[505 rows x 3 columns]


## Loading daily weather data

Creating a 'day' column that stores only the date values from 'time' column
(for linking weather dataset 'day' with daily dataset 'day')

In [8]:
# Load the weather dataset into a DataFrame
weather_daily_data = pd.read_csv(weather_daily_dataset_path)

# Convert the 'time' column to datetime format
weather_daily_data['time'] = pd.to_datetime(weather_daily_data['time'])

# Calculate the mean temperature for each day and store it in a new column 'mean_temp'
weather_daily_data['mean_temp'] = (weather_daily_data['temperatureMax'] + weather_daily_data['temperatureMin']) / 2

# Print the updated DataFrame
print(weather_daily_data)

     temperatureMax   temperatureMaxTime  windBearing                 icon  \
0             11.96  2011-11-11 23:00:00          123                  fog   
1              8.59  2011-12-11 14:00:00          198    partly-cloudy-day   
2             10.33  2011-12-27 02:00:00          225    partly-cloudy-day   
3              8.07  2011-12-02 23:00:00          232                 wind   
4              8.22  2011-12-24 23:00:00          252  partly-cloudy-night   
..              ...                  ...          ...                  ...   
877            9.03  2014-01-26 16:00:00          233    partly-cloudy-day   
878           10.31  2014-02-27 14:00:00          224    partly-cloudy-day   
879           18.97  2014-03-09 14:00:00          172  partly-cloudy-night   
880            8.83  2014-02-12 16:00:00          210                 wind   
881            9.90  2014-02-15 12:00:00          233                 wind   

     dewPoint   temperatureMinTime  cloudCover  windSpeed  pres

### Loading required data
Creating a DataFrame that have only the required data

In [9]:
# Create a new DataFrame with selected columns
selected_columns = ['mean_temp', 'pressure', 'humidity', 'windSpeed', 'time']
weather_selected = weather_daily_data[selected_columns]

# Print the new dataset
print(weather_selected)

     mean_temp  pressure  humidity  windSpeed       time
0       10.405   1016.08      0.95       3.88 2011-11-11
1        5.535   1007.71      0.88       3.94 2011-12-11
2        9.180   1032.76      0.74       3.54 2011-12-27
3        5.315   1012.12      0.87       3.00 2011-12-02
4        5.695   1028.17      0.80       4.46 2011-12-24
..         ...       ...       ...        ...        ...
877      6.145   1002.10      0.79       4.55 2014-01-26
878      7.120   1007.02      0.74       4.14 2014-02-27
879     13.310   1022.44      0.58       2.78 2014-03-09
880      5.930    994.27      0.75       7.24 2014-02-12
881      7.640    988.63      0.69       9.96 2014-02-15

[882 rows x 5 columns]


# Dataset with Household energy consumption values and weather values

## For all households

In [10]:
# Convert 'time' column in weather_selected to datetime objects
weather_selected['time'] = pd.to_datetime(weather_selected['time'])

# Create a list to store the merged data DataFrames
merged_data_list = []

# Iterate through each LCLid in energy_daily_selected
for lclid, data in grouped_data_selected.items():
    # Convert 'day' column in current LCLid data to datetime objects
    data['day'] = pd.to_datetime(data['day'])

    # Merge the current LCLid data with weather_selected based on the common date values
    merged_data_lclid = pd.merge(weather_selected, data, left_on='time', right_on='day', how='inner')

    # Drop the redundant 'day' column from the merged data
    merged_data_lclid.drop(columns=['day'], inplace=True)

    # Append the merged data to the merged_data_list
    merged_data_list.append(merged_data_lclid)

# Concatenate the merged data DataFrames in the list
merged_data = pd.concat(merged_data_list, ignore_index=True)

# Display the merged dataset
print(merged_data)


<ipython-input-10-fc90630300b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_selected['time'] = pd.to_datetime(weather_selected['time'])


         mean_temp  pressure  humidity  windSpeed       time      LCLid  \
0           10.000   1001.79      0.81       6.54 2012-11-25  MAC000002   
1           12.170   1008.74      0.90       5.74 2012-11-20  MAC000002   
2            7.830    979.63      0.85       4.07 2012-11-01  MAC000002   
3            5.295   1020.29      0.87       3.58 2012-11-06  MAC000002   
4            4.650   1001.72      0.80       5.63 2012-12-07  MAC000002   
...            ...       ...       ...        ...        ...        ...   
1542386      4.470   1001.76      0.91       1.52 2014-01-30  MAC005567   
1542387      6.145   1002.10      0.79       4.55 2014-01-26  MAC005567   
1542388      7.120   1007.02      0.74       4.14 2014-02-27  MAC005567   
1542389      5.930    994.27      0.75       7.24 2014-02-12  MAC005567   
1542390      7.640    988.63      0.69       9.96 2014-02-15  MAC005567   

         energy_sum  
0            10.545  
1            11.221  
2            12.209  
3          

## For one household

In [11]:
print(merged_data[merged_data['LCLid'] == "MAC003719"])

         mean_temp  pressure  humidity  windSpeed       time      LCLid  \
1015088     10.000   1001.79      0.81       6.54 2012-11-25  MAC003719   
1015089     12.170   1008.74      0.90       5.74 2012-11-20  MAC003719   
1015090      7.830    979.63      0.85       4.07 2012-11-01  MAC003719   
1015091      5.295   1020.29      0.87       3.58 2012-11-06  MAC003719   
1015092      4.650   1001.72      0.80       5.63 2012-12-07  MAC003719   
...            ...       ...       ...        ...        ...        ...   
1015361      4.470   1001.76      0.91       1.52 2014-01-30  MAC003719   
1015362      6.145   1002.10      0.79       4.55 2014-01-26  MAC003719   
1015363      7.120   1007.02      0.74       4.14 2014-02-27  MAC003719   
1015364      5.930    994.27      0.75       7.24 2014-02-12  MAC003719   
1015365      7.640    988.63      0.69       9.96 2014-02-15  MAC003719   

         energy_sum  
1015088      13.952  
1015089      12.864  
1015090      18.794  
1015091    

# Splitting dataset to training and testing sets



## Preprocessing dataset

### Counting amount of data for each LCLid (data available for each household)

In [12]:
# Convert 'time' column in weather_selected to datetime objects
weather_selected['time'] = pd.to_datetime(weather_selected['time'])

# Merge the data into merged_data DataFrame as described in your previous code

# Count the number of data rows for each LCLid
lclid_data_counts = merged_data['LCLid'].value_counts()

# Display the counts for each LCLid
print("Data row counts for each LCLid:")
print(lclid_data_counts)


Data row counts for each LCLid:
MAC000145    402
MAC000147    402
MAC000148    402
MAC000149    402
MAC000150    402
            ... 
MAC005560      1
MAC001150      1
MAC005563      1
MAC001957      1
MAC005565      1
Name: LCLid, Length: 5550, dtype: int64


<ipython-input-12-467dbfca5fe4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_selected['time'] = pd.to_datetime(weather_selected['time'])


### Identifiting the no of households with insufficient amount of data

In [13]:
# Count the number of data rows for each LCLid
lclid_data_counts = merged_data['LCLid'].value_counts()


# Get the total number of unique LCLid values
total_lclids = len(lclid_data_counts)

# Count the number of LCLid values with less than 100 data rows
count_less_than_100 = (lclid_data_counts < 100).sum()

# Display the count of LCLid values with less than 100 data rows
print("Number of LCLid values with less than 100 data rows:", count_less_than_100)

# Display the total number of unique LCLid values
print("Total number of unique LCLid values:", total_lclids)

Number of LCLid values with less than 100 data rows: 61
Total number of unique LCLid values: 5550


### Removing the households with less data, from the dataset

In [14]:
# Get the list of LCLid values with less than 100 data rows
lclids_to_remove = lclid_data_counts[lclid_data_counts < 100].index

# Remove rows corresponding to LCLid values with less than 100 data rows
filtered_data = merged_data[~merged_data['LCLid'].isin(lclids_to_remove)]

# also removing rows with NaN values in the dataset
filtered_data = filtered_data.dropna()

# Display the filtered data
print(filtered_data)

         mean_temp  pressure  humidity  windSpeed       time      LCLid  \
0           10.000   1001.79      0.81       6.54 2012-11-25  MAC000002   
1           12.170   1008.74      0.90       5.74 2012-11-20  MAC000002   
2            7.830    979.63      0.85       4.07 2012-11-01  MAC000002   
3            5.295   1020.29      0.87       3.58 2012-11-06  MAC000002   
4            4.650   1001.72      0.80       5.63 2012-12-07  MAC000002   
...            ...       ...       ...        ...        ...        ...   
1542386      4.470   1001.76      0.91       1.52 2014-01-30  MAC005567   
1542387      6.145   1002.10      0.79       4.55 2014-01-26  MAC005567   
1542388      7.120   1007.02      0.74       4.14 2014-02-27  MAC005567   
1542389      5.930    994.27      0.75       7.24 2014-02-12  MAC005567   
1542390      7.640    988.63      0.69       9.96 2014-02-15  MAC005567   

         energy_sum  
0            10.545  
1            11.221  
2            12.209  
3          

## Splitting the dataset to train and test
 where the split ratio(70:30) is made with every househould's data.

In [15]:
# Define the split percentages
train_percentage = 0.8 # 80% for training, 30% for testing
min_data_points = 10  # Minimum number of data points required for an LCLid

# Create a list to store DataFrames for training and testing
train_data_list = []
test_data_list = []

# Iterate through each unique LCLid and split the data based on train_percentage
unique_lclids = filtered_data['LCLid'].unique()
for lclid in unique_lclids:
    lclid_data = filtered_data[filtered_data['LCLid'] == lclid]

    # Check if there are sufficient data points for the current LCLid
    if len(lclid_data) >= min_data_points:
        # Split the data for the current LCLid into training and test sets
        train_data_lclid, test_data_lclid = train_test_split(lclid_data, train_size=train_percentage, shuffle=False)

        # Randomize the rows within each subset
        train_data_lclid = train_data_lclid.sample(frac=1, random_state=42)
        test_data_lclid = test_data_lclid.sample(frac=1, random_state=42)

        # Append to the train_data_list and test_data_list
        train_data_list.append(train_data_lclid)
        test_data_list.append(test_data_lclid)

# Concatenate the DataFrames in the lists
FL_train_set = pd.concat(train_data_list, ignore_index=True)
FL_test_set = pd.concat(test_data_list, ignore_index=True)

# Store the training and test sets in separate lists
FL_train_set_list = train_data_list
FL_test_sets_list = test_data_list

# Display the training and test sets
print("Training set:")
print(FL_train_set)
print("Test set:")
print(FL_test_set)

Training set:
         mean_temp  pressure  humidity  windSpeed       time      LCLid  \
0            6.915   1023.52      0.70       1.11 2013-02-16  MAC000002   
1            5.545   1031.32      0.91       1.35 2013-12-10  MAC000002   
2            8.930    998.27      0.91       1.29 2012-11-26  MAC000002   
3            8.065   1004.27      0.85       4.72 2013-12-31  MAC000002   
4            3.565    991.02      0.91       0.73 2013-03-18  MAC000002   
...            ...       ...       ...        ...        ...        ...   
1229099      5.915   1037.52      0.77       3.33 2013-11-25  MAC005567   
1229100      4.835   1029.69      0.80       4.25 2013-02-28  MAC005567   
1229101      6.525   1018.73      0.79       5.52 2013-12-20  MAC005567   
1229102      8.070   1006.71      0.87       2.13 2014-01-22  MAC005567   
1229103      2.340   1005.94      0.82       5.05 2013-03-10  MAC005567   

         energy_sum  
0            10.807  
1            12.882  
2             9.876

### For FL training, validation and test sets

In [16]:
# Display the first training set
print("First Training set:")
print(FL_train_set_list[0])
print("First Test set:")
print(FL_test_sets_list[0])

First Training set:
     mean_temp  pressure  humidity  windSpeed       time      LCLid  \
154      6.915   1023.52      0.70       1.11 2013-02-16  MAC000002   
215      5.545   1031.32      0.91       1.35 2013-12-10  MAC000002   
15       8.930    998.27      0.91       1.29 2012-11-26  MAC000002   
173      8.065   1004.27      0.85       4.72 2013-12-31  MAC000002   
158      3.565    991.02      0.91       0.73 2013-03-18  MAC000002   
..         ...       ...       ...        ...        ...        ...   
106      3.355   1023.08      0.78       4.86 2013-02-20  MAC000002   
14       5.745   1014.66      0.82       0.72 2012-11-18  MAC000002   
92      10.680   1012.62      0.86       8.91 2013-12-21  MAC000002   
179      3.065   1026.97      0.83       4.80 2013-02-25  MAC000002   
102      2.395   1010.88      0.63       3.52 2013-03-29  MAC000002   

     energy_sum  
154      10.807  
215      12.882  
15        9.876  
173      13.746  
158      13.295  
..          ...  
1

### Centralised training, validation and test sets

In [17]:
# Randomize the rows of filtered_data
filtered_data = filtered_data.sample(frac=1, random_state=42)

# Define the split percentages
train_percentage = 0.6  # 60% for training
val_percentage = 0.2    # 20% for validation
test_percentage = 0.2   # 20% for testing

# Split the data into training and temporary sets
Central_train_data, temp_data = train_test_split(filtered_data, train_size=train_percentage, random_state=42)

# Split the temporary data into validation and test sets
Central_val_data, Central_test_data = train_test_split(temp_data, train_size=val_percentage / (val_percentage + test_percentage), random_state=42)

# Display the sizes of the sets
print("Training set size:", len(Central_train_data))
print("Validation set size:", len(Central_val_data))
print("Test set size:", len(Central_test_data))


Training set size: 923463
Validation set size: 307821
Test set size: 307821


Displaying training, validation and test sets

In [18]:
# Display the training and test sets
print("Training set:")
print(Central_train_data)
print("Validation set:")
print(Central_val_data)
print("Test set:")
print(Central_test_data)


Training set:
         mean_temp  pressure  humidity  windSpeed       time      LCLid  \
1366339      8.170   1009.95      0.77       6.82 2012-12-30  MAC004957   
1219794      3.140   1012.97      0.83       4.76 2013-01-12  MAC004472   
152173       8.895   1016.76      0.89       4.78 2012-12-28  MAC000461   
622133       6.145   1002.10      0.79       4.55 2014-01-26  MAC002246   
435865       5.930    994.27      0.75       7.24 2014-02-12  MAC001539   
...            ...       ...       ...        ...        ...        ...   
394725       4.815   1010.13      0.89       2.19 2014-01-21  MAC001383   
1062413      8.530    996.96      0.93       2.20 2013-03-07  MAC003894   
533802       2.630   1018.79      0.86       0.68 2013-02-09  MAC001909   
715661       4.510   1006.64      0.75       6.04 2013-02-06  MAC002600   
1484893      5.895   1012.39      0.82       3.98 2014-01-23  MAC005356   

         energy_sum  
1366339       3.758  
1219794      11.290  
152173       17.431

# Centralised Training

## Defining features and target column

In [19]:
# Separate features and target variable
Central_X_train = Central_train_data[['mean_temp', 'pressure', 'humidity', 'windSpeed']]
Central_y_train = Central_train_data['energy_sum']

# Separate features and target variable
Central_X_val = Central_val_data[['mean_temp', 'pressure', 'humidity', 'windSpeed']]
Central_y_val = Central_val_data['energy_sum']

# Separate features and target variable
Central_X_test = Central_test_data[['mean_temp', 'pressure', 'humidity', 'windSpeed']]
Central_y_test = Central_test_data['energy_sum']


## Preprocessing (scaler and imputer)

In [20]:
# Define the datasets
datasets = [
    (Central_X_train, Central_y_train, "Central Training"),
    (Central_X_val, Central_y_val, "Central Validation"),
    (Central_X_test, Central_y_test, "Central Test")
]

# Preprocess each dataset
for X, y, name in datasets:
    # Impute missing values and scale features
    X_scaled = StandardScaler().fit_transform(SimpleImputer(strategy='mean').fit_transform(X))

    # Display dataset name and shape
    print(f"{name} Dataset:")
    print("Original Shape:", X.shape)
    print("Preprocessed Shape:", X_scaled.shape)
    print()

Central Training Dataset:
Original Shape: (923463, 4)
Preprocessed Shape: (923463, 4)

Central Validation Dataset:
Original Shape: (307821, 4)
Preprocessed Shape: (307821, 4)

Central Test Dataset:
Original Shape: (307821, 4)
Preprocessed Shape: (307821, 4)



In [21]:
# Check for missing values
print(Central_y_train.isnull().sum())
print(Central_X_train.isnull().sum())

# Replace missing values with mean (you can choose another strategy)
Central_y_train.fillna(Central_y_train.mean(), inplace=True)
Central_X_train.fillna(Central_X_train.mean(), inplace=True)

# Check for missing values
print(Central_y_train.isnull().sum())
print(Central_X_train.isnull().sum())

0
mean_temp    0
pressure     0
humidity     0
windSpeed    0
dtype: int64
0
mean_temp    0
pressure     0
humidity     0
windSpeed    0
dtype: int64


<ipython-input-21-e51efdbfd96c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Central_X_train.fillna(Central_X_train.mean(), inplace=True)
<ipython-input-21-e51efdbfd96c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Central_X_train.fillna(Central_X_train.mean(), inplace=True)
<ipython-input-21-e51efdbfd96c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

## Model selection

In [22]:

# # List of models to train
# models = [
#     LinearRegression(),
#     DecisionTreeRegressor(),
#     RandomForestRegressor(),
#     GradientBoostingRegressor(),
#     XGBRegressor(),
#     LGBMRegressor(),
#     CatBoostRegressor(),
#     SVR(),
#     MLPRegressor(),
#     KNeighborsRegressor()
# ]

# # Dictionary to store model performances
# model_performances = {}

# # Train and evaluate each model on the validation set
# for model in models:
#     model_name = model.__class__.__name__
#     print(f"Training {model_name}...")
#     model.fit(Central_X_train, Central_y_train)

#     # Predict on the validation set
#     y_pred_val = model.predict(Central_X_val)
#     # print(y_pred_val)

#     # Calculate Mean Squared Error
#     # mse_val = mean_squared_error(Central_y_val, y_pred_val)
#     model_performances[model_name] = (Central_y_val-y_pred_val)
#     print(model_performances[model_name])

# # Find the best-performing model
# best_model = min(model_performances, key=model_performances.get)
# print(f"Best-performing model: {best_model} (MSE: {model_performances[best_model]})")


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define models
linear_model = LinearRegression()
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
xgb_model = xgb.XGBRegressor(n_estimators=100, objective='reg:squarederror', random_state=0)

# Remove rows with missing values from both features and target variables
Central_train_data_cleaned = Central_train_data.dropna()
Central_X_train = Central_train_data_cleaned[['mean_temp', 'pressure', 'humidity', 'windSpeed']]
Central_y_train = Central_train_data_cleaned['energy_sum']
# Drop rows with NaN values in the target variable
Central_val_data.dropna(subset=['energy_sum'], inplace=True)


# Train linear model
linear_model.fit(Central_X_train, Central_y_train)

# Train random forest model
rf_model.fit(Central_X_train, Central_y_train)

# Train XGBoost model
xgb_model.fit(Central_X_train, Central_y_train)

# Build a simple feedforward neural network
model = keras.Sequential([
    layers.Input(shape=(4,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
model.fit(Central_X_train, Central_y_train, epochs=10, batch_size=32, validation_data=(Central_X_val, Central_y_val))


Epoch 1/10
28859/28859 [==============================] - 98s 3ms/step - loss: 125.0429 - mse: 125.0429 - val_loss: 116.8158 - val_mse: 116.8158
Epoch 2/10
28859/28859 [==============================] - 77s 3ms/step - loss: 118.2098 - mse: 118.2098 - val_loss: 116.5121 - val_mse: 116.5121
Epoch 3/10
28859/28859 [==============================] - 88s 3ms/step - loss: 118.0703 - mse: 118.0703 - val_loss: 116.4786 - val_mse: 116.4786
Epoch 4/10
28859/28859 [==============================] - 89s 3ms/step - loss: 118.0396 - mse: 118.0396 - val_loss: 116.7034 - val_mse: 116.7034
Epoch 5/10
28859/28859 [==============================] - 83s 3ms/step - loss: 118.0431 - mse: 118.0431 - val_loss: 116.5104 - val_mse: 116.5104
Epoch 6/10
28859/28859 [==============================] - 88s 3ms/step - loss: 118.0317 - mse: 118.0317 - val_loss: 116.4994 - val_mse: 116.4994
Epoch 7/10
28859/28859 [==============================] - 86s 3ms/step - loss: 118.0182 - mse: 118.0182 - val_loss: 116.6665 - val

### Defining the Evaluation function

In [39]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
def evaluate_model(model, X, y):
    predictions = model.predict(X)
    mse = mean_squared_error(y, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, predictions)
    r2 = r2_score(y, predictions)
    print(f"Model: {model.__class__.__name__}")
    print("Mean Squared Error:", mse)
    print("Root Mean Squared Error:", rmse)
    print("Mean Absolute Error:", mae)
    print("R-squared:", r2)
    print("R-squared variance:", r2*100)
    print()

models = [
    ("Linear Regression", linear_model),
    ('Random Forest', rf_model),
    ('XGBoost', xgb_model),
    ('Neural Network', model)
]


### Using the Evalution Function

In [40]:
# Define the models as trained earlier
models = [
    ('Linear Regression', linear_model),
    ('Random Forest', rf_model),
    ('XGBoost', xgb_model),
    ('Neural Network', model)
]

# Evaluate each model
for model_name, model in models:
    print(f'\n{model_name}:')
    evaluate_model(model, Central_X_val, Central_y_val)



Linear Regression:
Model: LinearRegression
Mean Squared Error: 116.41037279585728
Root Mean Squared Error: 10.789363873549602
Mean Absolute Error: 7.052658959584261
R-squared: 0.003915117595502782
R-squared variance: 0.39151175955027817


Random Forest:
Model: RandomForestRegressor
Mean Squared Error: 115.7289852389329
Root Mean Squared Error: 10.757740712572176
Mean Absolute Error: 7.013834106158011
R-squared: 0.009745524527547733
R-squared variance: 0.9745524527547733


XGBoost:
Model: XGBRegressor
Mean Squared Error: 115.72620195775852
Root Mean Squared Error: 10.7576113500051
Mean Absolute Error: 7.013739946677222
R-squared: 0.00976934013980546
R-squared variance: 0.976934013980546


Neural Network:
Model: RandomForestRegressor
Mean Squared Error: 115.7289852389329
Root Mean Squared Error: 10.757740712572176
Mean Absolute Error: 7.013834106158011
R-squared: 0.009745524527547733
R-squared variance: 0.9745524527547733



## Gated Convolution Neural Network (GCNN)

In [ ]:
# class GatedCNN24(nn.Module):
#     def __init__(self, input_channels, hidden_sizes):
#         super(GatedCNN24, self).__init__()
#         self.input_channels = input_channels
#         self.hidden_sizes = hidden_sizes

#         self.conv1 = nn.Conv2d(self.input_channels, self.hidden_sizes[0], kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(self.hidden_sizes[0], self.hidden_sizes[1], kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(self.hidden_sizes[1], self.hidden_sizes[2], kernel_size=3, padding=1)

#         self.gate_conv1 = nn.Conv2d(self.input_channels, self.hidden_sizes[0], kernel_size=3, padding=1)
#         self.gate_conv2 = nn.Conv2d(self.hidden_sizes[0], self.hidden_sizes[1], kernel_size=3, padding=1)
#         self.gate_conv3 = nn.Conv2d(self.hidden_sizes[1], self.hidden_sizes[2], kernel_size=3, padding=1)

#         self.fc1 = nn.Linear(self.hidden_sizes[2], 1)

#     def forward(self, x):
#         # Convolutional layers
#         conv1_out = self.conv1(x)
#         conv2_out = self.conv2(conv1_out)
#         conv3_out = self.conv3(conv2_out)

#         # Gating layers
#         gate_conv1_out = torch.sigmoid(self.gate_conv1(x))
#         gate_conv2_out = torch.sigmoid(self.gate_conv2(conv1_out))
#         gate_conv3_out = torch.sigmoid(self.gate_conv3(conv2_out))

#         # Element-wise multiplication (gating)
#         gated_conv1 = conv1_out * gate_conv1_out
#         gated_conv2 = conv2_out * gate_conv2_out
#         gated_conv3 = conv3_out * gate_conv3_out

#         # Sum along channels
#         gated_sum = gated_conv1 + gated_conv2 + gated_conv3

#         # Global average pooling
#         pooled = torch.mean(gated_sum, dim=[2, 3])

#         # Fully connected layer
#         output = self.fc1(pooled)
#         return output.squeeze()

# class EnergyDataset(Dataset):
#     def __init__(self, features, targets):
#         self.features = features
#         self.targets = targets

#     def __len__(self):
#         return len(self.features)

#     def __getitem__(self, idx):
#         inputs = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32)
#         target = torch.tensor(self.targets.iloc[idx], dtype=torch.float32)
#         return inputs, target

# # Define hyperparameters
# input_channels = 4  # Number of features
# hidden_sizes = [10, 8, 1]
# learning_rate = 0.001
# num_epochs = 10
# batch_size = 50

# # Create DataLoader for training
# train_dataset = EnergyDataset(Central_X_train, Central_y_train)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# # Create the model
# GatedCNN_model = GatedCNN24(input_channels, hidden_sizes)

# # Define loss function and optimizer
# criterion = nn.MSELoss()
# optimizer = optim.Adam(GatedCNN_model.parameters(), lr=learning_rate)

# # Training loop
# for epoch in range(num_epochs):
#     GatedCNN_model.train()  # Set the model in training mode
#     for inputs, targets in train_loader:
#         optimizer.zero_grad()

#         # Reshape the input tensor to match expected shape [batch_size, channels, height, width]
#         inputs = inputs.view(inputs.size(0), input_channels, 1, 1)

#         outputs = GatedCNN_model(inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


## Gated Neural Network model

In [ ]:
# # Build the Gated Neural Network model
# inputs = Input(shape=(4,))
# dense_layer = Dense(16, activation='relu')(inputs)

# # Gating mechanism
# dense_gate = Dense(16, activation='sigmoid')(inputs)
# gated_dense = Multiply()([dense_layer, dense_gate])

# output = Dense(1, activation='linear')(gated_dense)

# GNN_model = Model(inputs=inputs, outputs=output)

# # Compile the model
# GNN_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# # Train the model
# GNN_model.fit(Central_X_train, Central_y_train, epochs=10, batch_size=32, validation_split=0.2)

 a dataset, a model, a training function, and a test function.


Federated learning

Centralised training

Centralised testing